# So easy, *voilà*!

In this example notebook, we demonstrate how Voilà can render Jupyter notebooks with interactions requiring a roundtrip to the kernel.

## Jupyter Widgets

In [1]:
import ipywidgets as widgets

slider = widgets.FloatSlider(description='$x$')
text = widgets.FloatText(disabled=True, description='$x^2$')

def compute(*ignore):
    text.value = str(slider.value ** 2)

slider.observe(compute, 'value')

slider.value = 4

widgets.VBox([slider, text])

## Basic outputs of code cells

In [2]:
import pandas as pd

iris = pd.read_csv('https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv')
iris

/usr/local/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [3]:
import json

import numpy as np
import pandas as pd
from bqplot import DateScale, ColorScale
from py2vega.functions.type_coercing import toDate
from py2vega.functions.date_time import datetime

from ipydatagrid import Expr, DataGrid, TextRenderer, BarRenderer

n = 10000

df = pd.DataFrame(
    {
        "Value 1": np.random.randn(n),
        "Value 2": np.random.randn(n),
        "Dates": pd.date_range(end=pd.Timestamp("today"), periods=n),
    }
)

text_renderer = TextRenderer(
    text_color="black", background_color=ColorScale(min=-5, max=5)
)


def bar_color(cell):
    date = toDate(cell.value)
    return "green" if date > datetime("2000") else "red"


renderers = {
    "Value 1": text_renderer,
    "Value 2": text_renderer,
    "Dates": BarRenderer(
        bar_color=Expr(bar_color),
        format="%Y/%m/%d",
        format_type="time",
    ),
}

grid = DataGrid(df, base_row_size=30, base_column_size=300, renderers=renderers)
grid.transform([{"type": "sort", "columnIndex": 2, "desc": True}])
grid

DataGrid(auto_fit_params={'area': 'all', 'padding': 30, 'numCols': None}, base_column_size=300, base_row_size=…

## d3.js Example

In [1]:
%%html
<script src="https://requirejs.org/docs/release/2.3.6/minified/require.js"></script>

In [2]:
%%javascript
require.config({
    paths: {
        d3: 'https://d3js.org/d3.v5.min'
    }
});

<IPython.core.display.Javascript object>

In [3]:
%%javascript
(function(element) {
    require(['d3'], function(d3) {   
        var data = [1, 10, 4, 8, 6, 8, 4, 2, 1]

        var svg = d3.select(element.get(0)).append('svg')
            .attr('width', 400)
            .attr('height', 200);
        svg.selectAll('circle')
            .data(data)
            .enter()
            .append('circle')
            .attr("cx", function(d, i) {return 40 * (i + 1);})
            .attr("cy", function(d, i) {return 100 + 30 * (i % 3 - 1);})
            .style("fill", "#1570a4")
            .transition().duration(2000)
            .attr("r", function(d) {return 2*d;})
        ;
    })
})(element);

<IPython.core.display.Javascript object>

In [4]:
from IPython.core.display import display, HTML
from string import Template
import json

HTML('<script src="https://d3js.org/d3.v5.min.js"></script>')
css_text = '''
'''
js_text_template = Template('''
       var bogoSVG = d3.select("#$bogoanimation") 
          .append("svg")
          .attr("width", 300)
          .attr("height", 300);    

      var data = $python_data ;
       bogoSVG.append("circle")
          .style("stroke", "gray")
          .style("fill", "cyan")
          .attr("r", data[0]['r'])
          .attr("cx", data[0]['cx'])
          .attr("cy", data[0]['cy'])
          .transition()
             .delay(100)
             .duration(2000)  
             .attr("r", 10)
             .attr("cx", data[0]['cx'])
             .style("fill", "blue"); 
''')
html_template = Template('''
<style> $css_text </style>
<div id="animation"></div>
<script> $js_text </script>
''')
js_text = js_text_template.substitute({'python_data': json.dumps([{'r': 130, 'cx': 150, 'cy': 150}]),
                                       'bogoanimation': 'animation'})
HTML(html_template.substitute({'css_text': css_text, 'js_text': js_text}))